In [ ]:
test

In [ ]:

import os

# Option A: already set in your shell
# os.environ['OPENAI_API_KEY']

# Option B: set here for this notebook session
os.environ['OPENAI_API_KEY'] = "put open ai key here"

print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY set: True


In [3]:
!pip install langgraph


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [6]:
from typing import Literal
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode

from db_crawl_agents.llms.openai_integration.chat_openai import OpenAIChat
from db_crawl_agents.utils.runnable_chat_model import RunnableChatModel

# Define a tool
@tool
def get_weather(city: str) -> str:
    """Return a mock weather string for a city."""
    return f"It is 29°C and partly cloudy in {city}."

TOOLS = [get_weather]

# Provider-native tool schema (for OpenAI)
OPENAI_TOOLS = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather for a city (mock).",
        "parameters": {
            "type": "object",
            "properties": {"city": {"type":"string"}},
            "required": ["city"],
            "additionalProperties": False
        },
    },
}]

# Build the Runnable wrapping your ChatModel
base = OpenAIChat(model="gpt-4o")
llm = RunnableChatModel(base).bind_tools(OPENAI_TOOLS, tool_choice="auto")

# Graph nodes definition
def call_model(state: MessagesState):
    ai = llm.invoke(state["messages"])
    return {"messages": [ai]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    last = state["messages"][-1]
    if isinstance(last, AIMessage):
        tcs = (last.additional_kwargs or {}).get("tool_calls", [])
        if tcs:
            return "tools"
    return END

tool_node = ToolNode(TOOLS)

graph = StateGraph(MessagesState)
graph.add_node("model", call_model)
graph.add_node("tools", tool_node)

graph.add_edge(START, "model")
graph.add_conditional_edges("model", should_continue, {"tools": "tools", END: END})
graph.add_edge("tools", "model")

app = graph.compile()


# for chunk in app.stream(
#     {"messages": [HumanMessage(content="What's the weather in Bengaluru?")]},   # pass as positional
#     stream_mode="debug",
# ):
#     print( chunk)
# Run the graph
out = app.invoke({"messages": [HumanMessage(content="What's the weather in Bengaluru?")]})
final_ai = [m for m in out["messages"] if isinstance(m, AIMessage)][-1]
print("\nFINAL ANSWER:\n", final_ai)


FINAL ANSWER:
 content='' additional_kwargs={} response_metadata={} id='f7723e7b-4c29-4563-939d-4efabc0228ce' tool_calls=[{'name': 'get_weather', 'args': {'city': 'Bengaluru'}, 'id': 'call_a7zGrRVXviqCEKZI2NoWYYNS', 'type': 'tool_call'}]
